In [4]:
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
## Loading images and labels
(train_ds, train_labels), (test_ds, test_labels) = tfds.load(
    "tf_flowers",
    split=["train[:70%]", "train[:30%]"], ## Train test split
    batch_size=-1,
    as_supervised=True,  # Include labels
)

## Resizing images
train_ds = tf.image.resize(train_ds, (150, 150))
test_ds = tf.image.resize(test_ds, (150, 150))

## Transforming labels to correct format
train_labels = to_categorical(train_labels, num_classes=5)
test_labels = to_categorical(test_labels, num_classes=5)

In [2]:
!pip install tensorflow_datasets


     ---------------------------------------- 4.7/4.7 MB 3.5 MB/s eta 0:00:00
     -------------------------------------- 140.1/140.1 kB 1.2 MB/s eta 0:00:00
     -------------------------------------- 110.5/110.5 kB 1.3 MB/s eta 0:00:00
     -------------------------------------- 50.8/50.8 kB 653.4 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ------------------------------------ 211.7/211.7 kB 860.7 kB/s eta 0:00:00
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21500 sha256=fa8b1869d429753b80da1d427e72a849ab2eb4a1e9f1b9486e4f532cee729fa4
  Stored in directory: c:\users\ril\appdata\local\pip\cache\wheels\6a\fe\dc\a7b3e03dfd0afb3a19691905bbafac1fbaebb704a02a4daeb2
Successfully built promise


In [5]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

## Loading VGG16 model
base_model = VGG16(weights="imagenet", include_top=False, input_shape=train_ds[0].shape)
base_model.trainable = False ## Not trainable weights

## Preprocessing input
train_ds = preprocess_input(train_ds) 
test_ds = preprocess_input(test_ds)

58889256/58889256 [==============================] - 17s 0us/step


In [6]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [7]:
from tensorflow.keras import layers, models

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(50, activation='relu')
dense_layer_2 = layers.Dense(20, activation='relu')
prediction_layer = layers.Dense(5, activation='softmax')


model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)


es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=32, callbacks=[es])

Epoch 1/50
65/65 [==============================] - 206s 3s/step - loss: 1.7309 - accuracy: 0.5119 - val_loss: 1.1027 - val_accuracy: 0.5992
Epoch 2/50
65/65 [==============================] - 199s 3s/step - loss: 0.7771 - accuracy: 0.7377 - val_loss: 1.0058 - val_accuracy: 0.6576
Epoch 3/50
65/65 [==============================] - 200s 3s/step - loss: 0.5001 - accuracy: 0.8243 - val_loss: 0.9543 - val_accuracy: 0.6926
Epoch 4/50
65/65 [==============================] - 201s 3s/step - loss: 0.3663 - accuracy: 0.8774 - val_loss: 0.9962 - val_accuracy: 0.6770
Epoch 5/50
65/65 [==============================] - 196s 3s/step - loss: 0.2589 - accuracy: 0.9114 - val_loss: 0.9970 - val_accuracy: 0.7082
Epoch 6/50
29/65 [============>.................] - ETA: 1:29 - loss: 0.2003 - accuracy: 0.9386